In [1]:
import numpy as np
import pandas as pd

filename = 'patch_level_radiomics_feature_VTRPDAC_Test_PC3896_BL1_XX.csv'
# filename = 'VTRPDAC_Test_PC3896_BL1_XX.csv'
df = pd.read_csv(filename)

obj = {"img_width": df['image_width'].iloc[0],
       "img_height": df['image_height'].iloc[0],
       "patch_w": df['patch_width'].iloc[0],
       "patch_h": df['patch_height'].iloc[0],
       "png_w": df['image_width'].iloc[0] / df['patch_width'].iloc[0],
       "png_h": df['image_height'].iloc[0] / df['patch_height'].iloc[0]}

cols = list(df.columns)
modified = df[cols[5:7] + cols[11:12]]
modified.sort_values(['patch_x', 'patch_y'], ascending=[1, 1])
modified['i'] = modified['patch_x'] / df['patch_width']
modified['j'] = modified['patch_y'] / df['patch_height']
modified['n'] = modified['nuclei_ratio'] * 255
# .loc[row_indexer,col_indexer] = value

# modified.n = modified.n.round()
modified.i = np.ceil(modified.i).astype(int)
modified.j = np.ceil(modified.j).astype(int)
modified.n = np.ceil(modified.n).astype(int)

modified
modified.to_csv('modified.csv', index=False)

# df['nuclei_ratio']


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers